# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [1]:
import pandas as pd

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population


Read in the HTML from the ULR using the `requests` library.

In [2]:
import requests
URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(URL, headers=HEADERS)
response

<Response [200]>

Use Beautiful Soup to parse this string into a tree called `soup`

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [4]:
len(soup.find_all("table"))

10

We can use attributes like `class=` and `style=` to narrow down the list.

In [47]:
len(soup.find_all("table",
                  attrs={
                      "class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                      "style": "text-align:right"}
                  ))

1

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [48]:
table = soup.find_all("table",
                  attrs={
                      "class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                      "style": "text-align:right"}
                      )[0]

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [58]:
# initialize an empty list
rows = []

# iterate over all rows in the faculty table
for city in table.find_all("tr")[2:]:

    # Get all the cells (<td>) in the row.
    cells = city.find_all("td")


    city_name = cells[0].find("title") or cells[0]
    city = city_name.get_text(strip = True)

    state_name=cells[1].find("title") or cells[1]
    state=state_name.get_text(strip = True)

    population_num=cells[3].find("a") or cells[3]
    population=population_num.get_text(strip = True)

    land_area_num=cells[5].find("a") or cells[5]
    land_area=land_area_num.get_text(strip = True)

    rows.append({
        "city": city,
        "state": state,
        "2020 population": population,
        "2020 land area": land_area
    })

In [59]:
df=pd.DataFrame(rows)
df

,city,state,2020 population,2020 land area
0,New York[c],NY,"8,804,190",300.5
1,Los Angeles,CA,"3,898,747",469.5
2,Chicago,IL,"2,746,388",227.7
3,Houston,TX,"2,304,580",640.4
4,Phoenix,AZ,"1,608,139",518.0
...,...,...,...,...
341,Deltona,FL,"93,692",37.3
342,Federal Way,WA,"101,030",22.3
343,San Angelo,TX,"99,893",59.7
344,Tracy,CA,"93,000",25.9


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [60]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [63]:
import requests
URL = "https://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response2 = requests.get(URL, headers=HEADERS)
response2

<Response [200]>

In [64]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response2.content, "html.parser")

In [67]:
# initialize an empty list
rows2 = []

for course in soup.find_all("div", {"class": "courseblock"}):

    # Find the course name tag
    course_name_tag = course.find("p", {"class": "courseblocktitle"})
    course_name = course_name_tag.get_text(strip = True) if course_name_tag else "N/A"

    # Find the term offered tag (the first <p> with class noindent)
    term_offered_tag = course.find("p", {"class": "noindent"})
    term = term_offered_tag.get_text(strip = True) if term_offered_tag else "N/A"

    rows2.append({
        "course": course_name,
        "term": term
    })

In [69]:
df2=pd.DataFrame(rows2)
df2

,course,term
0,DATA 100. Data Science for All I.4 units,"Term Typically Offered: F, W, SP"
1,DATA 301. Introduction to Data Science.4 units,"Term Typically Offered: F, W, SP"
2,DATA 401. Data Science Process and Ethics.3 units,Term Typically Offered: F
3,DATA 402. Mathematical Foundations of Data Sci...,Term Typically Offered: F
4,DATA 403. Data Science Projects Laboratory.1 unit,Term Typically Offered: F
...,...,...
69,STAT 551. Statistical Learning with R.4 units,Term Typically Offered: F
70,STAT 566. Graduate Consulting Practicum.2 units,Term Typically Offered: SP
71,STAT 570. Selected Advanced Topics.1-4 units,Term Typically Offered: TBD
72,STAT 590. Graduate Seminar in Statistics.1 unit,"Term Typically Offered: F, W, SP"


Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag